# HazardCurvesGH.ipynb

## Load gauge time series from netCDF file and use to construct hazard curves

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.

:::{note}
DRAFT version now using preliminary results for 18 events.

Adapted from [](../LoadGaugesGH.ipynb)
:::

This notebook uses a datafile containing gauge time series the 18 ground motions developed for the 
Cascadia CoPes Hub, at gauge locations around Grays Harbor, for a 6-hour simulation using [GeoClaw](https://www.geoclaw.org) with 1/3 arcsecond resolution on the finest grid level.

This data file can be found from the Design Safe Jupyter Hub in
  ~/MyProjects/PRJ-6005/GraysHarborBridges
  
The gauge locations are on an interactive map that can be created in this notebook.

Data used can be found on DesignSafe in 

    ~/MyProjects/PRJ-6005/GraysHarborBridges
    
or downloaded to your local machine via:

    scp username@stampede3.tacc.utexas.edu:/corral/projects/NHERI/projects/7f2e74be-d7ca-4e0e-b69a-22c24840b078/GraysHarborBridges/*.nc ./
    

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import xarray
import os,sys,glob
from pathlib import Path
from IPython.display import FileLink

In [ ]:
sys.path.insert(0,'../../src')
import CHTuser

In [ ]:
from CHTuser import CHTtools

## Read the netCDF file containing all time series

Make sure `ncfile` includes the correct path to the netCDF file with gauge results.

In [ ]:
pwd

In [ ]:
if 0:
    # on Design Safe
    datadir = '~/MyProjects/PRJ-6005/GraysHarborBridges'
else:
    # local data
    datadir = '.'

# full path:
datadir = Path(datadir).resolve()

print(f'netcdf files in {datadir}:')
glob.glob(f'{datadir}/*.nc')

Set `location` to one of the locations in the directories above.

In [ ]:

location = 'OceanShores'
ncfile = f'{datadir}/{location}_gauges_18BuriedEvents.nc'

print(f'ncfile = {ncfile}')

In [ ]:
gauge_x, gauge_y, gauge_t, gauge_vals = CHTtools.read_allgauges_nc(ncfile)

## Convert to an xarray dataset

`gauge_vals` is one large 4-dimensional array indexed by the quantity of interest `qoi` as well as by `event`, `gaugeno`, and `time`.
For most purposes it's easiest  to convert is to an `xarray.Dataset`, a collection of DataArray's split up by the qoi:

In [ ]:
ds = gauge_vals.to_dataset(dim='qoi')

In [ ]:
all_events = ds.coords['event'].data
all_gauges = ds.coords['gaugeno'].data
times = ds.coords['time'].data

In [ ]:
print('\nEvents: \n',all_events)
print('\nGauges: \n',all_gauges)
print(f'\nAt {len(ds.coords['time'])} times up to {times.max()/3600} hours\n')

### Make an html file with interactive map showing all gauges:

Still need to clean up this function and move to `CHTtools`....

In [ ]:
def folium_plot_gauge(gaugenos, center=None, zoom_start=13):
    import folium
    import numpy as np

    if not isinstance(gaugenos, (list, tuple, np.ndarray)):
        # If it's not a list, wrap it in one
        gaugenos = [gaugenos]
       
    #tiles = 'OpenStreetMap'  # default tiles
    tiles = 'OpenTopoMap'   # show contours
    
    m = None

    for gaugeno in gaugenos:
 
        xg = gauge_x.sel(gaugeno=gaugeno)
        yg = gauge_y.sel(gaugeno=gaugeno)
        #print(f'Gauge %i is at (%.5f, %.5f)' % (gaugeno, xg, yg))

        if m is None:
            if center is None:
                location=(yg, xg)
            elif isinstance(center,int):
                xgc = gauge_x.sel(gaugeno=center)
                ygc = gauge_y.sel(gaugeno=center)
                location = (ygc, xgc)
            else:
                #assume center is (y,x) location
                location = center
            
            m = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
        
        folium.Marker(
            location=[yg,xg],
            #popup = f"<b>Gauge {gaugeno}</b>\n ({xg:.6f},\n{yg:.6f})",
            popup = f"<b>Gauge {gaugeno}</b>\n {yg:.6f}N\n{-xg:.6f}W",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
        
    return m

In [ ]:
all_gauges = ds.coords['gaugeno'].data
map_with_all_gauges = folium_plot_gauge(all_gauges, center=None, zoom_start=12)
fname = f'{location}_bridges_gauges_folium_map.html'
map_with_all_gauges.save(fname)
print('Created ',fname)

# to display the map here, uncomment the next line:
#map_with_all_gauges

FileLink(fname)

In [ ]:
# plot one gauge on interactive map

#mygaugeno = 431
#map = folium_plot_gauge(mygaugeno)  #  argument can be single gauge or list of gauges
#map

## Adding new qoi's

Now we can refer to `ds.h`, for example, to get the `h` variable (rather than having to use `gauge_vals.sel(qoi='h')`.

We can also easily add new quantities of interest to the dataset that are computed from others, e.g. the momentum flux.


In [ ]:
ds['momflux'] = ds.h * (ds.u**2 + ds.v**2)

In [ ]:
momflux_max = float(ds.momflux.max())
print(f'ds.momflux has shape {ds.momflux.shape}')
print(f'   and the maximum value over all events / gauges /times is {momflux_max:.2f} m^3/s^2')

In [ ]:
mygaugeno = 154
myevent = 'BL10D'

## Hazard curves

First define the weight for each event based on the logic tree, see [](GroundMotions).

In [ ]:
event_weights = {}

for event in ds.coords['event'].data:
    w = 1/3 * 0.5
    if 'B' in event:
        w *= 0.75
    else:
        w *= 0.25
    if 'D' in event:
        w *= 0.3
    elif 'M' in event:
        w *= 0.5
    else:
        w*=0.2
    event_weights[event] = w
    print(f'{event}:  {w:.5f}')

In [ ]:
def probs(hazard_qoi, exceed_vals, gaugeno, weights, events=None):
    if events is None:
        events = hazard_qoi.coords['event'].data  # all avaliable events
    pvals = zeros(exceed_vals.shape)
    for j in range(len(exceed_vals)):
        for eventk in events:
            if eventk not in hazard_qoi.coords['event']:
                raise ValueError(f'*** event {eventk} not available in hazard_qoi')
            if hazard_qoi.sel(gaugeno=gaugeno, event=eventk) > exceed_vals[j]:
                pk = event_weights[eventk]
                pvals[j] = pvals[j] + pk - pvals[j]*pk
    return pvals

### Quantity of interest

define `hmh0` to be the maximum of water depth `h` over all time minus `h0`, the initial water depth at the gauge.  Note that this creates a new data array `hmh0` that can be indexed by `gaugeno` and `event`.

In [ ]:
h0 = ds.h.sel(time=0)
hmh0 = ds.h.max(dim='time') - h0

In [ ]:
hmh0test = float(hmh0.sel(gaugeno=mygaugeno, event=myevent))
print(f'At gauge {mygaugeno}, event {myevent}, hmh0 = {hmh0test:.3f}')

In [ ]:
figure(figsize=(6,5))
ievents = range(len(ds.coords['event']))
hmax = hmh0.sel(gaugeno=mygaugeno)
plot(hmax, ievents)
yticks(ievents, hmax.coords['event'].data);
grid(True)
xlabel('meters')
title(f'Maximum h-h0 over {times.max()/3600} hours at Gauge {mygaugeno}');

### Create and plot a hazard curve

In [ ]:
exceed_vals = arange(0,9,0.2)
mygaugeno = 431
events = None # for all events
pvals = probs(hmh0, exceed_vals, mygaugeno, event_weights, events=None)
plot(exceed_vals,pvals,'r', label='All events')
grid(True)
legend(loc='upper right', framealpha=1)
title(f'Hazard curve of max(h-h0) for Gauge {mygaugeno}');
xlabel('exceedance value (m)')
ylabel('probability of exceedance')
savefig('sample_hazard_curve.png')

In [ ]:
if 0:
    for e,p in zip(exceed_vals,pvals):
        print(f'{e:.3f},  {p:.6f}')

## Hazard curves for maximum momentum flux

In [ ]:
max_momflux = ds.momflux.max(dim='time')
print(f'Over all gauges,events, maximum momentum flux is {max_momflux.data.max():.2f} m^3/s^2')

In [ ]:
figure(figsize=(6,5))
ievents = range(len(ds.coords['event']))
plot(max_momflux.sel(gaugeno=mygaugeno), ievents)
yticks(ievents, max_momflux.coords['event'].data);
grid(True)
xlabel('momentum flux (m^3/s^2)')
title(f'Maximum momentum flux over {times.max()/3600} hours at Gauge {mygaugeno}');

In [ ]:
exceed_vals = arange(0,200,10)
mygaugeno = 431
events = None # for all events
pvals = probs(max_momflux, exceed_vals, mygaugeno, event_weights, events=None)
plot(exceed_vals,pvals,'r', label='All events')
grid(True)
#xlim (0,15)
legend(loc='upper right', framealpha=1)
title(f'Hazard curve of max momentum flux for Gauge {mygaugeno}');
xlabel('exceedance value (m^3/s^2)')
ylabel('probability of exceedance')
savefig('sample_hazard_curve.png')